In [1]:
import asyncio
from copra.rest import Client as rClient 
import nest_asyncio
nest_asyncio.apply()

loop = asyncio.get_event_loop()

client = rClient(loop)

async def get_stats():
    btc_stats = await client.get_24hour_stats('BTC-USD')
    print(btc_stats)

loop.run_until_complete(get_stats())
loop.run_until_complete(client.close())

{'open': '48901.58', 'high': '49808.28', 'low': '42141', 'volume': '44458.80768029', 'last': '44978.13', 'volume_30day': '618672.21340678'}


In [2]:
import asyncio
from datetime import datetime, timedelta
import sys
import requests
from urllib.parse import urljoin 

import nest_asyncio
import utils
nest_asyncio.apply()

SYMBOL = 'BTC-USD'

from copra.websocket import Channel, Client

class Tick:

    def __init__(self, tick_dict):
        # self.product_id = tick_dict['product_id']
        # self.best_bid = float(tick_dict['best_bid'])
        # self.best_ask = float(tick_dict['best_ask'])
        self.price = float(tick_dict['price'])
        self.side = tick_dict['side']
        self.size = float(tick_dict['last_size'])
        self.time = datetime.strptime(tick_dict['time'], '%Y-%m-%dT%H:%M:%S.%fZ')

    def __repr__(self):
        return(f'{self.time} - price: {self.price:.2f} vol: {self.size} side: {self.side}')


class Ticker(Client):

    def on_message(self, message):
        if message['type'] == 'ticker' and 'time' in message:
            tick = Tick(message)
            print(tick)



loop = asyncio.get_event_loop()

channel = Channel('ticker', SYMBOL)

ticker = Ticker(loop, channel)

try:
    loop.run_forever()
except KeyboardInterrupt:
    loop.run_until_complete(ticker.close())
    loop.close()

2021-05-17 08:18:35.201052 - price: 44884.69 vol: 0.0596679 side: sell
2021-05-17 08:18:35.808382 - price: 44884.70 vol: 0.00063725 side: buy
2021-05-17 08:18:35.869988 - price: 44884.69 vol: 0.0133 side: sell
2021-05-17 08:18:35.869988 - price: 44884.68 vol: 0.0131 side: sell
2021-05-17 08:18:37.155755 - price: 44877.59 vol: 0.01 side: sell
2021-05-17 08:18:37.155755 - price: 44877.59 vol: 0.0131 side: sell
2021-05-17 08:18:37.155755 - price: 44877.58 vol: 0.0131 side: sell
2021-05-17 08:18:37.155755 - price: 44870.68 vol: 0.03311951 side: sell
2021-05-17 08:18:37.534166 - price: 44870.69 vol: 0.00033262 side: buy
2021-05-17 08:18:37.839749 - price: 44870.68 vol: 0.001496 side: sell
2021-05-17 08:18:38.515401 - price: 44870.75 vol: 0.11081038 side: buy
2021-05-17 08:18:38.523634 - price: 44870.75 vol: 0.00115265 side: buy
2021-05-17 08:18:38.531882 - price: 44870.74 vol: 0.02329667 side: sell
2021-05-17 08:18:38.919628 - price: 44870.75 vol: 0.00212764 side: buy
2021-05-17 08:18:38.94

RuntimeError: Cannot close a running event loop

In [ ]:
import cbpro 
public_client = cbpro.PublicClient()


public_client.get_product_order_book(SYMBOL)

In [28]:
COINBASE_DATETIME_FORMAT = '%Y-%m-%dT%H:%M:%S'

now = datetime.utcnow()
# delta = timedelta(minutes=1)

#current_date = now.date()
# current_time = now.time()


now_plus1 = now + timedelta(minutes=1)
# after_1min_date = datetime.combine(current_date, after_1min)

# time_after_1min = datetime.combine(current_time + timedelta(minutes=1)).time()
print('now:', now)
print('now + 1 min:', now_plus1)

now_str = utils.dt_to_str(now, COINBASE_DATETIME_FORMAT)
now_plus1_str = utils.dt_to_str(now_plus1, COINBASE_DATETIME_FORMAT)

print('now:', now_str)
print('now + 1 min:', now_plus1_str)


result = public_client.get_product_historic_rates('BTC-USD', granularity=60, start=now_str, end=now_plus1_str)
# [ time, low, high, open, close, volume ]

print(result)
# print(utils.ts_to_dt(result[0][0], 'korea'))
# print(result[0])

now: 2021-05-12 10:18:25.382856
now + 1 min: 2021-05-12 10:19:25.382856
now: 2021-05-12T10:18:25
now + 1 min: 2021-05-12T10:19:25
[]


In [29]:
start = now_str
stop = now_plus1_str

        
COINBASE_API_URL = 'https://api.pro.coinbase.com/'
PATH = 'products/BTC-USD/candles'

print('Fetching candles between: ', start, '...', stop)

params = {
    'start': start,
    'stop': stop,
    'granularity': 60
}

response = requests.get(urljoin(COINBASE_API_URL, PATH), params=params).json()
print(response)

Fetching candles between:  2021-05-12T10:18:25 ... 2021-05-12T10:19:25
[[1620814680, 56400.01, 56419.73, 56419.72, 56400.01, 1.26064699], [1620814620, 56404.73, 56489.46, 56472.06, 56409.8, 6.73364177], [1620814560, 56353.87, 56482.19, 56353.87, 56472.06, 13.0339258], [1620814500, 56351.24, 56406.93, 56400, 56354.45, 18.87199003], [1620814440, 56400, 56490.4, 56471.63, 56400, 12.53549498], [1620814380, 56461.55, 56534.6, 56534.6, 56471.63, 14.01686192], [1620814320, 56521.28, 56571.61, 56566, 56534.6, 11.33562854], [1620814260, 56566, 56650.63, 56573.74, 56566, 11.89362383], [1620814200, 56532.71, 56609.5, 56609.5, 56566.34, 12.92887179], [1620814140, 56554.79, 56613.45, 56559.45, 56603.66, 11.73410831], [1620814080, 56508.11, 56579.17, 56513.72, 56559.45, 15.5438121], [1620814020, 56404.87, 56521.34, 56449.12, 56502.34, 11.48502419], [1620813960, 56419.02, 56585.06, 56585.06, 56449.12, 21.42069197], [1620813900, 56523.68, 56624.75, 56548.39, 56585.06, 30.11783206], [1620813840, 56470.